In [24]:
import numpy as np
import mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

(60000, 28, 28)
(60000,)


In [25]:
###
seed = 1
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.1, random_state=seed)
print(x_train.shape)
print(x_val.shape) 
print(y_train.shape)
print(y_val.shape) 

(54000, 28, 28)
(6000, 28, 28)
(54000,)
(6000,)


In [26]:
###
# Normalize the images.
x_train = (x_train / 255) - 0.5
x_val = (x_val / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
x_train = x_train.reshape((-1, 784))
x_val = x_val.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(x_train.shape) # (60000, 784)
print(x_val.shape)  # (6000, 784)
print(test_images.shape)  # (10000, 784)

(54000, 784)
(6000, 784)
(10000, 784)


In [3]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 784)
(10000, 784)


In [52]:
# 0.97 -> 0.94 / 2. layer 64 -> 10
model = Sequential([
  Dense(128, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(10, activation='softmax'),
])

In [72]:
model.compile(
  optimizer=Adam(learning_rate=0.001),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [5]:
### Early Stopping
# monitor=accuracy, patience=8, min_delta=0.001 => l:0.2085 a:0.9609
# monitor=accuracy, patience=10, min_delta=0.001 => l:0.2298 a:0.9524 / l:0.2944 a:0.9583
# monitor=accuracy, patience=8 => l:0.3120 a:9551
# monitor=accuracy, patience=10 => l:0.2984 a:9592
# monitor=loss, patience=10 => l:0.3277 a:0.9567

In [73]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=15, 
    mode='min'
)

In [74]:
# batch_size 32->64 / l:0.3277 a:0.9567 -> l:0.3574 a:0.9617
model.fit(
  x_train, # training data
  to_categorical(y_train), # training targets
  epochs=1000,
  batch_size=128,
  validation_data=(x_val, to_categorical(y_val)),
  callbacks=[early_stopping],
)

Epoch 1/1000
422/422 [==============================] - 1s 3ms/step - loss: 0.0250 - accuracy: 0.9930 - val_loss: 0.1325 - val_accuracy: 0.9743
Epoch 2/1000
422/422 [==============================] - 1s 3ms/step - loss: 0.0198 - accuracy: 0.9939 - val_loss: 0.1731 - val_accuracy: 0.9678
Epoch 3/1000
422/422 [==============================] - 1s 2ms/step - loss: 0.0229 - accuracy: 0.9934 - val_loss: 0.1374 - val_accuracy: 0.9753
Epoch 4/1000
422/422 [==============================] - 1s 3ms/step - loss: 0.0236 - accuracy: 0.9931 - val_loss: 0.1442 - val_accuracy: 0.9727
Epoch 5/1000
422/422 [==============================] - 1s 3ms/step - loss: 0.0162 - accuracy: 0.9952 - val_loss: 0.1696 - val_accuracy: 0.9727
Epoch 6/1000
422/422 [==============================] - 1s 2ms/step - loss: 0.0166 - accuracy: 0.9950 - val_loss: 0.1613 - val_accuracy: 0.9730
Epoch 7/1000
422/422 [==============================] - 1s 2ms/step - loss: 0.0216 - accuracy: 0.9937 - val_loss: 0.1402 - val_accuracy:

In [75]:
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

313/313 [==============================] - 0s 1ms/step - loss: 0.1404 - accuracy: 0.9784


[0.14040715992450714, 0.9783999919891357]

In [76]:
Y_pred = model.predict(test_images)
y_pred = np.argmax(Y_pred, axis=1)
print(accuracy_score(test_labels, y_pred))
print(Y_pred)
print(y_pred)

0.9784
[[4.10397723e-23 8.60691241e-10 4.90234210e-13 ... 1.00000000e+00
  4.58654709e-17 7.57239688e-11]
 [1.13170359e-11 2.47241108e-14 1.00000000e+00 ... 3.00583386e-12
  3.56023747e-18 4.07627523e-19]
 [2.23274402e-11 9.99987602e-01 9.55964266e-11 ... 1.03386728e-05
  1.52134328e-06 2.05552695e-08]
 ...
 [2.50193055e-19 8.46339397e-14 9.80392131e-17 ... 2.45265132e-14
  1.53710473e-16 1.21795855e-11]
 [1.43210927e-18 0.00000000e+00 1.68124462e-24 ... 2.24243252e-20
  1.17655028e-14 1.43245061e-22]
 [1.39666413e-24 1.88298251e-25 0.00000000e+00 ... 0.00000000e+00
  4.52824075e-31 9.65861923e-34]]
[7 2 1 ... 4 5 6]


In [77]:
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.97      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.98      0.97      0.98       982
           5       0.97      0.98      0.98       892
           6       0.98      0.98      0.98       958
           7       0.96      0.99      0.97      1028
           8       0.97      0.98      0.97       974
           9       0.98      0.95      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
